# Extracting Undiagnosed Conditions from Clinical Notes

## Medicare Risk Adjustment: 
In the United States, the Centers for Medicare & Medicaid Services sets reimbursement for private Medicare plan sponsors based on the assessed risk of their beneficiaries. Information found in unstructured medical records may be more indicative of member risk than existing structured data, creating more accurate risk pools.

#### Initial configurations

In [ ]:
import os
import json
import string
import numpy as np
import pandas as pd

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel
from sparknlp.training import CoNLL

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.0
sparknlp_jsl.version : 3.4.0
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.0 
 Master 
 spark://10.139.64.6:7077 
 AppName 
 Databricks Shell

## Download oncology notes

In this notebook we will use the transcribed medical reports in [www.mtsamples.com](www.mtsamples.com). 

You can download those reports by the script [here](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/databricks/python/healthcare_case_studies/mt_scrapper.py).
    
We will use slightly modified version of some clinical notes which are downloaded from [www.mtsamples.com](www.mtsamples.com).

Let's create the folder which we will store the notes.

In [ ]:
notes_path='/FileStore/HLS/nlp/data/'
delta_path='/FileStore/HLS/nlp/delta/jsl/'

dbutils.fs.mkdirs(notes_path)
os.environ['notes_path']=f'/dbfs{notes_path}'

In [ ]:
%sh
cd $notes_path
wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_oncology_10.zip
unzip -o mt_oncology_10.zip

In [ ]:
dbutils.fs.ls(f'{notes_path}/mt_oncology_10')

## Read Data and Write to Bronze Delta Layer

There are 50 clinical notes stored in delta table. We read the data and write the raw notes data into bronze delta tables

In [ ]:
df = sc.wholeTextFiles(f'{notes_path}/mt_oncology_10/mt_note_0*.txt').toDF().withColumnRenamed('_1','path').withColumnRenamed('_2','text')
display(df.limit(5))

path text dbfs:/FileStore/HLS/nlp/data/mt_oncology_10/mt_note_01.txt 
Medical Specialty:Hematology - Oncology
Sample Name: BRCA-2 mutation
Description: Discharge summary of a patient with a BRCA-2 mutation.
(Medical Transcription Sample Report)
DISCHARGE DIAGNOSES: BRCA-2 mutation.
HISTORY OF PRESENT ILLNESS: The patient is a 59-year-old with a BRCA-2 mutation. Her sister died of breast cancer at age 32 and her daughter had breast cancer at age 27.
PHYSICAL EXAMINATION: The chest was clear. The abdomen was nontender. Pelvic examination shows no masses. No heart murmur.
HOSPITAL COURSE: The patient underwent surgery on the day of admission. In the postoperative course she was afebrile and unremarkable. The patient regained bowel function and was discharged on the morning of the fourth postoperative day.
OPERATIONS AND PROCEDURES: Total abdominal hysterectomy/bilateral salpingo-oophorectomy with resection of ovarian fossa peritoneum en bloc on July 25, 2006.
PATHOLOGY: A 105-gram uterus without dysplasia or cancer.
CONDITION ON DISCHARGE: Stable.
PLAN: The patient will remain at rest initially with progressive ambulation after. She will avoid lifting, driving or intercourse. She will call me if any fevers, drainage, bleeding, or pain. Follow up in my office in four weeks. Family history, social history, psychosocial needs per the social worker.
DISCHARGE MEDICATIONS: Percocet 5 #40 one every 3 hours p.r.n. pain.
 dbfs:/FileStore/HLS/nlp/data/mt_oncology_10/mt_note_02.txt Medical Specialty:Hematology - Oncology
Sample Name: Mullerian Adenosarcoma
Description: Discharge summary of a patient presenting with a large mass aborted through the cervix.
(Medical Transcription Sample Report)
PRINCIPAL DIAGNOSIS: Mullerian adenosarcoma.
HISTORY OF PRESENT ILLNESS: The patient is a 56-year-old presenting with a large mass aborted through the cervix.
PHYSICAL EXAM:CHEST: Clear. There is no heart murmur. ABDOMEN: Nontender.
PELVIC: There is a large mass in the vagina.
HOSPITAL COURSE: The patient went to surgery on the day of admission. The postoperative course was marked by fever and ileus. The patient regained bowel function. She was discharged on the morning of the seventh postoperative day.
OPERATIONS: July 25, 2006: Total abdominal hysterectomy, bilateral salpingo-oophorectomy.
DISCHARGE CONDITION: Stable.
PLAN: The patient will remain at rest initially with progressive ambulation thereafter. She will avoid lifting, driving, stairs, or intercourse. She will call me for fevers, drainage, bleeding, or pain. Family history, social history, and psychosocial needs per the social worker. The patient will follow up in my office in one week.
PATHOLOGY: Mullerian adenosarcoma.
MEDICATIONS: Percocet 5, #40, one q.3 h. p.r.n. pain.
 dbfs:/FileStore/HLS/nlp/data/mt_oncology_10/mt_note_03.txt Medical Specialty:Hematology - Oncology
Sample Name: Leiomyosarcoma
Description: Discharge summary of patient with leiomyosarcoma and history of pulmonary embolism, subdural hematoma, pancytopenia, and pneumonia.
(Medical Transcription Sample Report)
ADMITTING DIAGNOSES:1. Leiomyosarcoma.2. History of pulmonary embolism.3. History of subdural hematoma.4. Pancytopenia.5. History of pneumonia.
PROCEDURES DURING HOSPITALIZATION:1. Cycle six of CIVI-CAD (Cytoxan, Adriamycin, and DTIC) from 07/22/2008 to 07/29/2008.2. CTA, chest PE study showing no evidence for pulmonary embolism.
3. Head CT showing no evidence of acute intracranial abnormalities.4. Sinus CT, normal mini-CT of the paranasal sinuses.
HISTORY OF PRESENT ILLNESS: Ms. ABC is a pleasant 66-year-old Caucasian female who first palpated a mass in the left posterior arm in spring of 2007. The mass increased in size and she was seen by her primary care physician and referred to orthopedic surgeon. MRI showed inflammation and was thought to be secondary to rheumatoid arthritis. The mass increased in size. She eventually underwent a partial resection found to have pathologic grade 2 leiomyosarcoma, margins were 

In [ ]:
df.write.format('delta').mode('overwrite').save(f'{delta_path}/bronze/mt-oc-notes')
display(dbutils.fs.ls(f'{delta_path}/bronze/mt-oc-notes'))

path,name,size
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/_delta_log/,_delta_log/,0
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00000-629290f6-ec4f-4b1d-af88-c0f766613839-c000.snappy.parquet,part-00000-629290f6-ec4f-4b1d-af88-c0f766613839-c000.snappy.parquet,14428
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00000-75e2d6f1-6687-4013-9cf3-2cfe3527d119-c000.snappy.parquet,part-00000-75e2d6f1-6687-4013-9cf3-2cfe3527d119-c000.snappy.parquet,51666
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00000-95d8840c-c4ee-4da6-81cf-c67c52a57c34-c000.snappy.parquet,part-00000-95d8840c-c4ee-4da6-81cf-c67c52a57c34-c000.snappy.parquet,16827
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00000-aafafba8-7fe0-4199-b48e-e1eb3865943b-c000.snappy.parquet,part-00000-aafafba8-7fe0-4199-b48e-e1eb3865943b-c000.snappy.parquet,16863
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00000-b209709a-428b-40fd-b432-615e7ef69bfc-c000.snappy.parquet,part-00000-b209709a-428b-40fd-b432-615e7ef69bfc-c000.snappy.parquet,14428
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00001-0b9fd0ea-b414-4c8a-9e99-b2b8503ec440-c000.snappy.parquet,part-00001-0b9fd0ea-b414-4c8a-9e99-b2b8503ec440-c000.snappy.parquet,8850
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00001-4eade9bf-7bc4-4efa-9cca-51094baf55c2-c000.snappy.parquet,part-00001-4eade9bf-7bc4-4efa-9cca-51094baf55c2-c000.snappy.parquet,8886
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00001-6eb87dd7-d2d3-46c6-b949-0055ac0c40c8-c000.snappy.parquet,part-00001-6eb87dd7-d2d3-46c6-b949-0055ac0c40c8-c000.snappy.parquet,51356
dbfs:/FileStore/HLS/nlp/delta/jsl/bronze/mt-oc-notes/part-00001-bb70970c-31e4-46af-aed3-a87b6729984d-c000.snappy.parquet,part-00001-bb70970c-31e4-46af-aed3-a87b6729984d-c000.snappy.parquet,8951


In [ ]:
sample_text = df.limit(3).select("text").collect()[0][0]
print(sample_text)

Medical Specialty:Hematology - Oncology
Sample Name: BRCA-2 mutation
Description: Discharge summary of a patient with a BRCA-2 mutation.
(Medical Transcription Sample Report)
DISCHARGE DIAGNOSES: BRCA-2 mutation.
HISTORY OF PRESENT ILLNESS: The patient is a 59-year-old with a BRCA-2 mutation. Her sister died of breast cancer at age 32 and her daughter had breast cancer at age 27.
PHYSICAL EXAMINATION: The chest was clear. The abdomen was nontender. Pelvic examination shows no masses. No heart murmur.
HOSPITAL COURSE: The patient underwent surgery on the day of admission. In the postoperative course she was afebrile and unremarkable. The patient regained bowel function and was discharged on the morning of the fourth postoperative day.
OPERATIONS AND PROCEDURES: Total abdominal hysterectomy/bilateral salpingo-oophorectomy with resection of ovarian fossa peritoneum en bloc on July 25, 2006.
PATHOLOGY: A 105-gram uterus without dysplasia or cancer.
CONDITION ON DISCHARGE: Stable.
PLAN: The patient will remain at rest initially with progressive ambulation after. She will avoid lifting, driving or intercourse. She will call me if any fevers, drainage, bleeding, or pain. Follow up in my office in four weeks. Family history, social history, psychosocial needs per the social worker.
DISCHARGE MEDICATIONS: Percocet 5 #40 one every 3 hours p.r.n. pain.

## ICD-10 code extraction
Now, we will create a pipeline to extract ICD10 codes. This pipeline will find diseases and problems and then map their ICD10 codes. We will also check if this problem is still present or not.

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")
 
sentenceDetector = SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")
 
tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\
 
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")
 
c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 
 
clinical_ner = MedicalNerModel.pretrained("ner_jsl_enriched", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")
 
ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(["Oncological", "Disease_Syndrome_Disorder", "Heart_Disease"])
 
sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")
 
icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models")\
    .setInputCols(["ner_chunk", "sbert_embeddings"])\
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")\
    .setReturnCosineDistances(True)
 
clinical_assertion = AssertionDLModel.pretrained("jsl_assertion_wip", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
 
resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver,
        clinical_assertion
    ])
 
data_ner = spark.createDataFrame([[""]]).toDF("text")
 
icd_model = resolver_pipeline.fit(data_ner)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_jsl_enriched download started this may take some time.
Approximate size to download 14.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
Approximate size to download 1.4 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
jsl_assertion_wip download started this may take some time.
Approximate size to download 1.4 MB
[ | ][ / ][OK!]

We can transform the data. In path column, we have long path. Instead we will use filename column. Every file name refers to different patient.

In [ ]:
path_array = F.split(df['path'], '/')
df = df.withColumn('filename', path_array.getItem(F.size(path_array)- 1)).select(['filename', 'text'])

icd10_sdf = icd_model.transform(df)

/databricks/spark/python/pyspark/sql/column.py:327: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
 warnings.warn(

Let's see how our model extracted ICD Codes on a sample.

In [ ]:
light_model = LightPipeline(icd_model)

light_result = light_model.fullAnnotate(sample_text)

from sparknlp_display import EntityResolverVisualizer

vis = EntityResolverVisualizer()

# Change color of an entity label
vis.set_label_colors({'PROBLEM':'#008080'})

icd_vis = vis.display(light_result[0], 'ner_chunk', 'icd10cm_code', return_html=True)

displayHTML(icd_vis)

Medical Specialty:Hematology - Oncology Sample Name: BRCA-2 mutation Description: Discharge summary of a patient with a BRCA-2 mutation. (Medical Transcription Sample Report) DISCHARGE DIAGNOSES: BRCA-2 mutation. HISTORY OF PRESENT ILLNESS: The patient is a 59-year-old with a BRCA-2 mutation. Her sister died of breast cancer Oncological C5092 breast cancer at age 32 and her daughter had breast cancer Oncological C5092 breast cancer at age 27. PHYSICAL EXAMINATION: The chest was clear. The abdomen was nontender. Pelvic examination shows no masses. No heart murmur. HOSPITAL COURSE: The patient underwent surgery on the day of admission. In the postoperative course she was afebrile and unremarkable. The patient regained bowel function and was discharged on the morning of the fourth postoperative day. OPERATIONS AND PROCEDURES: Total abdominal hysterectomy/bilateral salpingo-oophorectomy with resection of ovarian fossa peritoneum en bloc on July 25, 2006. PATHOLOGY: A 105-gram uterus without dysplasia Disease_Syndrome_Disorder P614 diaphyseal dysplasia or cancer Oncological C801 cancer . CONDITION ON DISCHARGE: Stable. PLAN: The patient will remain at rest initially with progressive ambulation after. She will avoid lifting, driving or intercourse. She will call me if any fevers, drainage, bleeding, or pain. Follow up in my office in four weeks. Family history, social history, psychosocial needs per the social worker. DISCHARGE MEDICATIONS: Percocet 5 #40 one every 3 hours p.r.n. pain.

ICD resolver can also tell us HCC status. HCC status is 1 if the Medicare Risk Adjusment model contains ICD code.

In [ ]:
icd10_hcc_df = icd10_sdf.select("filename", F.explode(F.arrays_zip(icd10_sdf.ner_chunk.result, 
                                                                   icd10_sdf.icd10cm_code.result,
                                                                   icd10_sdf.icd10cm_code.metadata,
                                                                   icd10_sdf.assertion.result
                                                                  )).alias("cols")) \
                            .select("filename", F.expr("cols['0']").alias("chunk"),
                                    F.expr("cols['1']").alias("icd10_code"),
                                    F.expr("cols['2']['all_k_aux_labels']").alias("hcc_list"),
                                    F.expr("cols['3']").alias("assertion")
                                   ).toPandas()

In [ ]:
icd10_hcc_df.head()

Out[19]:

,filename,chunk,icd10_code,hcc_list,assertion
0,mt_note_01.txt,breast cancer,C5092,0||1||12:::0||1||12:::1||0||0:::0||0||0:::1||0||0:::1||0||0:::0||0||0:::0||1||12,Family
1,mt_note_01.txt,breast cancer,C5092,0||1||12:::0||1||12:::1||0||0:::0||0||0:::1||0||0:::1||0||0:::0||0||0:::0||1||12,Family
2,mt_note_01.txt,dysplasia,P614,1||0||0:::1||0||0:::1||0||0:::1||0||0:::1||0||0:::1||0||0:::1||0||0:::1||0||0:::1||0||0:::1||0||...,Absent
3,mt_note_01.txt,cancer,C801,1||1||12:::1||1||10:::1||0||0:::1||0||0:::1||1||8:::1||0||0:::1||0||0:::1||0||0:::1||1||11:::1||...,Absent
4,mt_note_02.txt,Adenosarcoma,C541,1||1||12:::1||1||12:::1||1||11:::0||0||0:::0||0||0:::1||0||0:::0||0||0:::0||0||0:::1||0||0:::1||...,Present


In [ ]:
icd10_hcc_df["hcc_status"] = icd10_hcc_df["hcc_list"].apply(lambda x: x.split("||")[1])
icd10_df = icd10_hcc_df.drop("hcc_list", axis = 1)
icd10_df.head()

Out[20]:

,filename,chunk,icd10_code,assertion,hcc_status
0,mt_note_01.txt,breast cancer,C5092,Family,1
1,mt_note_01.txt,breast cancer,C5092,Family,1
2,mt_note_01.txt,dysplasia,P614,Absent,0
3,mt_note_01.txt,cancer,C801,Absent,1
4,mt_note_02.txt,Adenosarcoma,C541,Present,1


In [ ]:
icd10_df = icd10_df[icd10_df.hcc_status=="1"]
icd10_df = icd10_df[~icd10_df.assertion.isin(["Family", "Past"])][['filename','chunk','icd10_code']].drop_duplicates()

We filtered the ICD codes based on HCC status. Now, we will create an ICD_code list column

In [ ]:
icd10_df['Extracted_Entities_vs_ICD_Codes'] = list(zip(icd10_df.chunk, icd10_df.icd10_code))
icd10_df.head(10)

Out[22]:

,filename,chunk,icd10_code,Extracted_Entities_vs_ICD_Codes
3,mt_note_01.txt,cancer,C801,"(cancer, C801)"
4,mt_note_02.txt,Adenosarcoma,C541,"(Adenosarcoma, C541)"
6,mt_note_02.txt,ileus,K5669,"(ileus, K5669)"
8,mt_note_03.txt,leiomyosarcoma,C499,"(leiomyosarcoma, C499)"
12,mt_note_03.txt,Leiomyosarcoma,C499,"(Leiomyosarcoma, C499)"
14,mt_note_03.txt,Pancytopenia,D6181,"(Pancytopenia, D6181)"
18,mt_note_03.txt,rheumatoid arthritis,M069,"(rheumatoid arthritis, M069)"
29,mt_note_03.txt,cancer,C801,"(cancer, C801)"
46,mt_note_05.txt,Breast Cancer,C5092,"(Breast Cancer, C5092)"
47,mt_note_05.txt,ductal carcinoma of the left breast,C5091,"(ductal carcinoma of the left breast, C5091)"


In [ ]:
icd10_codes= icd10_df.groupby("filename").icd10_code.apply(lambda x: list(x)).reset_index()
icd10_vs_entities = icd10_df.groupby("filename").Extracted_Entities_vs_ICD_Codes.apply(lambda x: list(x)).reset_index()

icd10_df_all = icd10_codes.merge(icd10_vs_entities)

icd10_df_all

Out[23]:

,filename,icd10_code,Extracted_Entities_vs_ICD_Codes
0,mt_note_01.txt,[C801],"[(cancer, C801)]"
1,mt_note_02.txt,"[C541, K5669]","[(Adenosarcoma, C541), (ileus, K5669)]"
2,mt_note_03.txt,"[C499, C499, D6181, M069, C801]","[(leiomyosarcoma, C499), (Leiomyosarcoma, C499), (Pancytopenia, D6181), (rheumatoid arthritis, M..."
3,mt_note_05.txt,"[C5092, C5091, C779, C5092, C800, G20, C5092]","[(Breast Cancer, C5092), (ductal carcinoma of the left breast, C5091), (T1c, Nx, M0, C779), (bre..."
4,mt_note_06.txt,[F319],"[(type 1 bipolar disease, F319)]"
5,mt_note_07.txt,[C459],"[(Description: Biopsy-proven mesothelioma, C459)]"
6,mt_note_08.txt,"[C459, C800]","[(malignant mesothelioma, C459), (metastatic disease, C800)]"
7,mt_note_09.txt,"[D5702, K5505]","[(Sickle cell crisis, D5702), (Veno-occlusive crisis, K5505)]"


## Gender Classification

In Spark NLP, we have a pretrained model to detect gender of patient. Let's use it by `ClassifierDLModel`

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

tokenizer = Tokenizer()\
      .setInputCols(["document"])\
      .setOutputCol("token")\

biobert_embeddings = BertEmbeddings().pretrained('biobert_pubmed_base_cased') \
        .setInputCols(["document",'token'])\
        .setOutputCol("bert_embeddings")

sentence_embeddings = SentenceEmbeddings() \
     .setInputCols(["document", "bert_embeddings"]) \
     .setOutputCol("sentence_bert_embeddings") \
     .setPoolingStrategy("AVERAGE")

genderClassifier = ClassifierDLModel.pretrained('classifierdl_gender_biobert', 'en', 'clinical/models') \
       .setInputCols(["document", "sentence_bert_embeddings"]) \
       .setOutputCol("gender")

gender_pipeline = Pipeline(stages=[documentAssembler,
                                   #sentenceDetector,
                                   tokenizer, 
                                   biobert_embeddings, 
                                   sentence_embeddings, 
                                   genderClassifier])

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
classifierdl_gender_biobert download started this may take some time.
Approximate size to download 21 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [ ]:
data_ner = spark.createDataFrame([[""]]).toDF("text")

gender_model = gender_pipeline.fit(data_ner)

gender_df = gender_model.transform(df)

In [ ]:
gender_pd_df = gender_df.select("filename", F.explode(F.arrays_zip(gender_df.gender.result,
                                                                   gender_df.gender.metadata)).alias("cols")) \
                       .select("filename",
                               F.expr("cols['0']").alias("Gender"),
                               F.expr("cols['1']['Female']").alias("Female"),
                               F.expr("cols['1']['Male']").alias("Male")).toPandas()

gender_pd_df['Gender'] = gender_pd_df.apply(lambda x : "F" if float(x['Female']) >= float(x['Male']) else "M", axis=1)

gender_pd_df = gender_pd_df[['filename', 'Gender']]

All patients' gender is ready in a dataframe.

In [ ]:
gender_pd_df

Out[27]:

,filename,Gender
0,mt_note_01.txt,F
1,mt_note_02.txt,F
2,mt_note_03.txt,F
3,mt_note_04.txt,F
4,mt_note_05.txt,F
5,mt_note_06.txt,F
6,mt_note_07.txt,F
7,mt_note_08.txt,F
8,mt_note_09.txt,F


## Age

We can get patient's age from the notes by another pipeline. We are creating an age pipeline to get `AGE` labelled entities. In a note, more than one age entity can be extracted. We will get the first age entity as patient's age.

In [ ]:
date_ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(["Age"])

age_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        date_ner_converter
    ])

data_ner = spark.createDataFrame([[""]]).toDF("text")

age_model = age_pipeline.fit(data_ner)

In [ ]:
light_model = LightPipeline(age_model)

light_result = light_model.fullAnnotate(sample_text)

from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

Medical Specialty:Hematology - Oncology Sample Name: BRCA-2 mutation Description: Discharge summary of a patient with a BRCA-2 mutation. (Medical Transcription Sample Report) DISCHARGE DIAGNOSES: BRCA-2 mutation. HISTORY OF PRESENT ILLNESS: The patient is a 59-year-old Age with a BRCA-2 mutation. Her sister died of breast cancer at age 32 Age and her daughter had breast cancer at age 27 Age . PHYSICAL EXAMINATION: The chest was clear. The abdomen was nontender. Pelvic examination shows no masses. No heart murmur. HOSPITAL COURSE: The patient underwent surgery on the day of admission. In the postoperative course she was afebrile and unremarkable. The patient regained bowel function and was discharged on the morning of the fourth postoperative day. OPERATIONS AND PROCEDURES: Total abdominal hysterectomy/bilateral salpingo-oophorectomy with resection of ovarian fossa peritoneum en bloc on July 25, 2006. PATHOLOGY: A 105-gram uterus without dysplasia or cancer. CONDITION ON DISCHARGE: Stable. PLAN: The patient will remain at rest initially with progressive ambulation after. She will avoid lifting, driving or intercourse. She will call me if any fevers, drainage, bleeding, or pain. Follow up in my office in four weeks. Family history, social history, psychosocial needs per the social worker. DISCHARGE MEDICATIONS: Percocet 5 #40 one every 3 hours p.r.n. pain.

In [ ]:
age_result = age_model.transform(df)

age_df = age_result.select("filename",F.explode(F.arrays_zip(age_result.ner_chunk.result,
                                                             age_result.ner_chunk.metadata)).alias("cols")) \
                   .select("filename", 
                           F.expr("cols['0']").alias("Age"),
                           F.expr("cols['1']['entity']").alias("ner_label")).toPandas().groupby('filename').first().reset_index()

In [ ]:
age_df.head()

Out[31]:

,filename,Age,ner_label
0,mt_note_01.txt,59-year-old,Age
1,mt_note_02.txt,56-year-old,Age
2,mt_note_03.txt,66-year-old,Age
3,mt_note_04.txt,61-year-old,Age
4,mt_note_05.txt,57-year-old,Age


In [ ]:
age_df.Age = age_df.Age.replace(r"\D", "", regex = True).astype(int)
age_df.drop('ner_label', axis=1, inplace=True)
age_df.head()

Out[32]:

,filename,Age
0,mt_note_01.txt,59
1,mt_note_02.txt,56
2,mt_note_03.txt,66
3,mt_note_04.txt,61
4,mt_note_05.txt,57


# Calculating Medicare Risk Adjusment Score

Now, we have all data which can be extracted from clinical notes. Now we can calculate Medicare Risk Adjusment Score.

In [ ]:
patient_df = age_df.merge(icd10_df_all, on='filename', how = "left")\
                   .merge(gender_pd_df, on='filename', how = "left").dropna()

#patient_df.icd10_code = patient_df.icd10_code.fillna("[]")

There are more parameters used to calculate risk adjusment score. But those parameters may not be extracted from clinical notes. Let's add them manually.

In [ ]:
imported_details = pd.DataFrame({"filename" : ['mt_note_01.txt', 'mt_note_03.txt', 'mt_note_05.txt', 
                                               'mt_note_06.txt', 'mt_note_08.txt', 'mt_note_09.txt'],
                                 "eligibility" : ["CFA", "CND", "CPA", "CFA", "CND", "CPA"],
                                 "orec" : ["0", "1", "3", "0", "1", "3"],
                                 "medicaid":[True, False, True, True, False, True]
                      })

patient_df = patient_df.merge(imported_details)

In [ ]:
patient_df.head()

Out[38]:

,filename,Age,icd10_code,Extracted_Entities_vs_ICD_Codes,Gender,eligibility,orec,medicaid
0,mt_note_01.txt,59,[C801],"[(cancer, C801)]",F,CFA,0,True
1,mt_note_03.txt,66,"[C499, C499, D6181, M069, C801]","[(leiomyosarcoma, C499), (Leiomyosarcoma, C499), (Pancytopenia, D6181), (rheumatoid arthritis, M...",F,CND,1,False
2,mt_note_05.txt,57,"[C5092, C5091, C779, C5092, C800, G20, C5092]","[(Breast Cancer, C5092), (ductal carcinoma of the left breast, C5091), (T1c, Nx, M0, C779), (bre...",F,CPA,3,True
3,mt_note_06.txt,63,[F319],"[(type 1 bipolar disease, F319)]",F,CFA,0,True
4,mt_note_08.txt,66,"[C459, C800]","[(malignant mesothelioma, C459), (metastatic disease, C800)]",F,CND,1,False


In [ ]:
patient_sdf = spark.createDataFrame(patient_df[['filename','Age', 'icd10_code', 'Gender', 'eligibility', 'orec', 'medicaid']])
patient_sdf.show()

+--------------+---+--------------------+------+-----------+----+--------+
 filename|Age| icd10_code|Gender|eligibility|orec|medicaid|
+--------------+---+--------------------+------+-----------+----+--------+
mt_note_01.txt| 59| [C801]| F| CFA| 0| true|
mt_note_03.txt| 66|[C499, C499, D618...| F| CND| 1| false|
mt_note_05.txt| 57|[C5092, C5091, C7...| F| CPA| 3| true|
mt_note_06.txt| 63| [F319]| F| CFA| 0| true|
mt_note_08.txt| 66| [C459, C800]| F| CND| 1| false|
mt_note_09.txt| 19| [D5702, K5505]| F| CPA| 3| true|
+--------------+---+--------------------+------+-----------+----+--------+

In [ ]:
from pyspark.sql.types import MapType, IntegerType, DoubleType, StringType, StructType, StructField, FloatType
import pyspark.sql.functions as f

schema = StructType([
            StructField('risk_score', FloatType()),
            StructField('hcc_lst', StringType()),
            StructField('parameters', StringType()),
            StructField('details', StringType())])

In [ ]:
from sparknlp_jsl.functions import profile

In [ ]:
df = patient_sdf.withColumn("hcc_profile", profile(patient_sdf.icd10_code, 
                                                   patient_sdf.Age, 
                                                   patient_sdf.Gender,
                                                   patient_sdf.eligibility,
                                                   patient_sdf.orec,
                                                   patient_sdf.medicaid
                                                  ))

In [ ]:
df.show()

+--------------+---+--------------------+------+-----------+----+--------+--------------------+
 filename|Age| icd10_code|Gender|eligibility|orec|medicaid| hcc_profile|
+--------------+---+--------------------+------+-----------+----+--------+--------------------+
mt_note_01.txt| 59| [C801]| F| CFA| 0| true|{"hcc_lst":["D1",...|
mt_note_03.txt| 66|[C499, C499, D618...| F| CND| 1| false|{"hcc_lst":["D2",...|
mt_note_05.txt| 57|[C5092, C5091, C7...| F| CPA| 3| true|{"hcc_lst":["HCC7...|
mt_note_06.txt| 63| [F319]| F| CFA| 0| true|{"hcc_lst":["D1",...|
mt_note_08.txt| 66| [C459, C800]| F| CND| 1| false|{"hcc_lst":["HCC8...|
mt_note_09.txt| 19| [D5702, K5505]| F| CPA| 3| true|{"hcc_lst":["D1",...|
+--------------+---+--------------------+------+-----------+----+--------+--------------------+

In [ ]:
df = df.withColumn("hcc_profile", F.from_json(F.col("hcc_profile"), schema))
df= df.withColumn("risk_score", df.hcc_profile.getItem("risk_score"))\
      .withColumn("hcc_lst", df.hcc_profile.getItem("hcc_lst"))\
      .withColumn("parameters", df.hcc_profile.getItem("parameters"))\
      .withColumn("details", df.hcc_profile.getItem("details"))

In [ ]:
df.select('filename','risk_score','icd10_code', 'Age', 'Gender', 'eligibility', 'orec', 'medicaid').show(truncate=False )

+--------------+----------+---------------------------------------------+---+------+-----------+----+--------+
filename |risk_score|icd10_code |Age|Gender|eligibility|orec|medicaid|
+--------------+----------+---------------------------------------------+---+------+-----------+----+--------+
mt_note_01.txt|0.158 |[C801] |59 |F |CFA |0 |true |
mt_note_03.txt|1.03 |[C499, C499, D6181, M069, C801] |66 |F |CND |1 |false |
mt_note_05.txt|2.991 |[C5092, C5091, C779, C5092, C800, G20, C5092]|57 |F |CPA |3 |true |
mt_note_06.txt|0.299 |[F319] |63 |F |CFA |0 |true |
mt_note_08.txt|2.714 |[C459, C800] |66 |F |CND |1 |false |
mt_note_09.txt|1.234 |[D5702, K5505] |19 |F |CPA |3 |true |
+--------------+----------+---------------------------------------------+---+------+-----------+----+--------+

In [ ]:
df.show(truncate=100, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------
 filename | mt_note_01.txt 
 Age | 59 
 icd10_code | [C801] 
 Gender | F 
 eligibility | CFA 
 orec | 0 
 medicaid | true 
 hcc_profile | {0.158, ["D1","HCC12"], {"elig":"CFA","age":59,"sex":"F","origds":false,"disabled":false,"medicai... 
 risk_score | 0.158 
 hcc_lst | ["D1","HCC12"] 
 parameters | {"elig":"CFA","age":59,"sex":"F","origds":false,"disabled":false,"medicaid":true} 
 details | {"CFA_D1":0.0,"CFA_HCC12":0.158} 
-RECORD 1-----------------------------------------------------------------------------------------------------------
 filename | mt_note_03.txt 
 Age | 66 
 icd10_code | [C499, C499, D6181, M069, C801] 
 Gender | F 
 eligibility | CND 
 orec | 1 
 medicaid | false 
 hcc_profile | {1.03, ["D2","HCC10","HCC40"], {"elig":"CND","age":66,"sex":"F","origds":true,"disabled":false,"m... 
 risk_score | 1.03 
 hcc_lst | ["D2","HCC10","HCC40"] 
 parameters | {"elig":"CND","age":66,"sex":"F","origds":true,"disabled":false,"medicaid":false} 
 details | {"CND_OriginallyDisabled_Female":0.0,"CND_D2":0.0,"CND_HCC10":0.663,"CND_HCC40":0.367} 
-RECORD 2-----------------------------------------------------------------------------------------------------------
 filename | mt_note_05.txt 
 Age | 57 
 icd10_code | [C5092, C5091, C779, C5092, C800, G20, C5092] 
 Gender | F 
 eligibility | CPA 
 orec | 3 
 medicaid | true 
 hcc_profile | {2.991, ["HCC78","D2","HCC8"], {"elig":"CPA","age":57,"sex":"F","origds":false,"disabled":true,"m... 
 risk_score | 2.991 
 hcc_lst | ["HCC78","D2","HCC8"] 
 parameters | {"elig":"CPA","age":57,"sex":"F","origds":false,"disabled":true,"medicaid":true} 
 details | {"CPA_HCC78":0.536,"CPA_D2":0.0,"CPA_HCC8":2.455} 
-RECORD 3-----------------------------------------------------------------------------------------------------------
 filename | mt_note_06.txt 
 Age | 63 
 icd10_code | [F319] 
 Gender | F 
 eligibility | CFA 
 orec | 0 
 medicaid | true 
 hcc_profile | {0.299, ["D1","HCC59"], {"elig":"CFA","age":63,"sex":"F","origds":false,"disabled":false,"medicai... 
 risk_score | 0.299 
 hcc_lst | ["D1","HCC59"] 
 parameters | {"elig":"CFA","age":63,"sex":"F","origds":false,"disabled":false,"medicaid":true} 
 details | {"CFA_D1":0.0,"CFA_HCC59":0.299} 
-RECORD 4-----------------------------------------------------------------------------------------------------------
 filename | mt_note_08.txt 
 Age | 66 
 icd10_code | [C459, C800] 
 Gender | F 
 eligibility | CND 
 orec | 1 
 medicaid | false 
 hcc_profile | {2.714, ["HCC8","D1"], {"elig":"CND","age":66,"sex":"F","origds":true,"disabled":false,"medicaid"... 
 risk_score | 2.714 
 hcc_lst | ["HCC8","D1"] 
 parameters | {"elig":"CND","age":66,"sex":"F","origds":true,"disabled":false,"medicaid":false} 
 details | {"CND_OriginallyDisabled_Female":0.0,"CND_HCC8":2.714,"CND_D1":0.0} 
-RECORD 5-----------------------------------------------------------------------------------------------------------
 filename | mt_note_09.txt 
 Age | 19 
 icd10_code | [D5702, K5505] 
 Gender | F 
 eligibility | CPA 
 orec | 3 
 medicaid | true 
 hcc_profile | {1.234, ["D1","HCC46"], {"elig":"CPA","age":19,"sex":"F","origds":false,"disabled":true,"medicaid... 
 risk_score | 1.234 
 hcc_lst | ["D1","HCC46"] 
 parameters | {"elig":"CPA","age":19,"sex":"F","origds":false,"disabled":true,"medicaid":true} 
 details | {"CPA_D1":0.0,"CPA_HCC46":1.234}

Now, we have risk score of each patient!

## License
Copyright / License info of the notebook. Copyright [2021] the Notebook Authors.  The source in this notebook is provided subject to the [Apache 2.0 License](https://spdx.org/licenses/Apache-2.0.html).  All included or referenced third party libraries are subject to the licenses set forth below.

|Library Name|Library License|Library License URL|Library Source URL|
| :-: | :-:| :-: | :-:|
|Pandas |BSD 3-Clause License| https://github.com/pandas-dev/pandas/blob/master/LICENSE | https://github.com/pandas-dev/pandas|
|Numpy |BSD 3-Clause License| https://github.com/numpy/numpy/blob/main/LICENSE.txt | https://github.com/numpy/numpy|
|Apache Spark |Apache License 2.0| https://github.com/apache/spark/blob/master/LICENSE | https://github.com/apache/spark/tree/master/python/pyspark|
|BeautifulSoup|MIT License|https://www.crummy.com/software/BeautifulSoup/#Download|https://www.crummy.com/software/BeautifulSoup/bs4/download/|
|Requests|Apache License 2.0|https://github.com/psf/requests/blob/main/LICENSE|https://github.com/psf/requests|
|Spark NLP Display|Apache License 2.0|https://github.com/JohnSnowLabs/spark-nlp-display/blob/main/LICENSE|https://github.com/JohnSnowLabs/spark-nlp-display|
|Spark NLP |Apache License 2.0| https://github.com/JohnSnowLabs/spark-nlp/blob/master/LICENSE | https://github.com/JohnSnowLabs/spark-nlp|
|Spark NLP for Healthcare|[Proprietary license - John Snow Labs Inc.](https://www.johnsnowlabs.com/spark-nlp-health/) |NA|NA|



|Author|
|-|
|Databricks Inc.|
|John Snow Labs Inc.|

## Disclaimers
Databricks Inc. (“Databricks”) does not dispense medical, diagnosis, or treatment advice. This Solution Accelerator (“tool”) is for informational purposes only and may not be used as a substitute for professional medical advice, treatment, or diagnosis. This tool may not be used within Databricks to process Protected Health Information (“PHI”) as defined in the Health Insurance Portability and Accountability Act of 1996, unless you have executed with Databricks a contract that allows for processing PHI, an accompanying Business Associate Agreement (BAA), and are running this notebook within a HIPAA Account.  Please note that if you run this notebook within Azure Databricks, your contract with Microsoft applies.